In [1]:
import sys
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install ipcalc
!{sys.executable} -m pip install graphviz
!{sys.executable} -m pip install xgboost

# import all necessary packages
import matplotlib.pyplot as plt # plotting
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import ipcalc

In [62]:
# Data cleaning and enhancing

# read data
nRows = 10000
data = pd.read_csv('newNIMS.csv', sep= ',', header= 0, nrows = nRows)

# remove the columns which have no unique elements
data = data[[col for col in data if data[col].nunique() > 1]]

# Convert the columns of non numerical type to numerical type
#data['Flow.ID_factor'] = data['Flow.ID'].factorize()[0]
#data['Flow.Duration_factor'] = data['Flow.Duration'].factorize()[0]
#data['ProtocolName_factor'] = data['ProtocolName'].factorize()[0]
#data['Timestamp_factor'] = data['Timestamp'].factorize()[0]
#for i in range(1,32):
#    sip=[]
#    dip=[]
#    print("Calculating all network address for /"+str(i))
#    for j in range(0,nRows):
#        SourceIP = data['Source.IP'][j]+"/"+str(i)
#        DestinationIP = data['Destination.IP'][j]+"/"+str(i)
#        Sadd = ipcalc.Network(SourceIP)
#        Dadd = ipcalc.Network(DestinationIP)
#        sip.append(str(Sadd.network()))
#        dip.append(str(Dadd.network()))
#    data['SourceIP_'+str(i)] = pd.Series(sip)
#    data['SourceIP_'+str(i)+'_factor'] = data['SourceIP_'+str(i)].factorize()[0] 
#    data['DestinationIP_'+str(i)] = pd.Series(dip)
#    data['DestinationIP_'+str(i)+'_factor'] = data['DestinationIP_'+str(i)].factorize()[0] 

In [63]:
data.columns.values

array(['min_fpktl', 'mean_fpktl', 'max_fpktl', 'std_fpktl', 'min_bpktl',
       'mean_bpktl', 'max_bpktl', 'std_bpktl', 'min_fiat', 'mean_fiat',
       'max_fiat', 'std_fiat', 'min_biat', 'mean_biat', 'max_biat',
       'std_biat', 'duration', 'proto', 'total_fpackets', 'total_fvolume',
       'total_bpackets', 'total_bvolume', 'class'], dtype=object)

In [64]:
data.head()

,min_fpktl,mean_fpktl,max_fpktl,std_fpktl,min_bpktl,mean_bpktl,max_bpktl,std_bpktl,min_fiat,mean_fiat,...,mean_biat,max_biat,std_biat,duration,proto,total_fpackets,total_fvolume,total_bpackets,total_bvolume,class
0,40,45,50,3,40,48,113,18,361,38823,...,20317,58785,20354,7241033,6,12,546,15,727,TELNET
1,40,46,56,4,40,60,98,17,324,21859,...,16003,129174,33664,240455,6,12,553,16,975,FTP
2,52,226,460,207,52,890,1500,607,152,17430,...,15731,90358,27594,6870746,6,14,3166,14,12473,HTTP
3,56,62,64,4,76,170,213,64,235202,235202,...,5723,5723,0,78872373,17,4,248,4,683,DNS
4,101,101,101,0,195,412,634,115,1528,65552,...,65386,428798,160266,262790503,17,21,2121,21,8659,lime


In [67]:
# Reindex the dataframe such that 1st column is target column
'''
x = ['L7Protocol', 'Flow.ID_factor', 'DestinationIP_1_factor', 'DestinationIP_10_factor' ,'DestinationIP_11_factor' ,'DestinationIP_12_factor' ,
     'DestinationIP_13_factor' ,'DestinationIP_14_factor' ,'DestinationIP_15_factor' ,'DestinationIP_16_factor' ,'DestinationIP_17_factor' ,'DestinationIP_18_factor' ,
     'DestinationIP_19_factor' ,'DestinationIP_2_factor' ,'DestinationIP_20_factor' ,'DestinationIP_21_factor' ,'DestinationIP_22_factor' ,'DestinationIP_23_factor' ,
     'DestinationIP_24_factor' ,'DestinationIP_25_factor' ,'DestinationIP_26_factor' ,'DestinationIP_27_factor' ,'DestinationIP_28_factor' ,'DestinationIP_29_factor' ,
     'DestinationIP_3_factor' ,'DestinationIP_30_factor' ,'DestinationIP_31_factor' ,'DestinationIP_4_factor' ,'DestinationIP_5_factor' ,'DestinationIP_6_factor' ,
     'DestinationIP_7_factor' ,'DestinationIP_8_factor' ,'DestinationIP_9_factor' ,'SourceIP_10_factor' ,'SourceIP_11_factor' ,'SourceIP_12_factor' ,
     'SourceIP_13_factor' ,'SourceIP_14_factor' ,'SourceIP_15_factor' ,'SourceIP_16_factor' ,'SourceIP_17_factor' ,'SourceIP_18_factor' ,'SourceIP_19_factor' ,
     'SourceIP_2_factor' ,'SourceIP_20_factor' ,'SourceIP_21_factor' ,'SourceIP_22_factor' ,'SourceIP_23_factor' ,'SourceIP_24_factor' ,'SourceIP_25_factor' ,
     'SourceIP_26_factor' ,'SourceIP_27_factor' ,'SourceIP_28_factor' ,'SourceIP_29_factor' ,'SourceIP_3_factor' ,'SourceIP_30_factor' ,'SourceIP_31_factor' ,
     'SourceIP_4_factor' ,'SourceIP_5_factor' ,'SourceIP_6_factor' ,'SourceIP_7_factor' ,'SourceIP_8_factor' ,'SourceIP_9_factor' ,'SourceIP_1_factor'  'Source.Port',
     'Destination.Port', 'Timestamp_factor', 'Flow.Duration_factor','Total.Fwd.Packets', 'Total.Backward.Packets', 'Total.Length.of.Fwd.Packets', 
     'Total.Length.of.Bwd.Packets', 'Fwd.Packet.Length.Max', 'Fwd.Packet.Length.Min', 'Fwd.Packet.Length.Mean', 'Fwd.Packet.Length.Std', 'Bwd.Packet.Length.Max', 
     'Bwd.Packet.Length.Min', 'Bwd.Packet.Length.Mean','Bwd.Packet.Length.Std', 'Flow.Bytes.s', 'Flow.Packets.s','Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max',
     'Flow.IAT.Min', 'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std', 'Fwd.IAT.Max', 'Fwd.IAT.Min', 'Bwd.IAT.Total', 'Bwd.IAT.Mean', 'Bwd.IAT.Std',  'Bwd.IAT.Max', 
     'Bwd.IAT.Min', 'Fwd.PSH.Flags', 'Fwd.Header.Length', 'Bwd.Header.Length', 'Fwd.Packets.s', 'Bwd.Packets.s','Min.Packet.Length', 'Max.Packet.Length', 
     'Packet.Length.Mean', 'Packet.Length.Std', 'Packet.Length.Variance', 'FIN.Flag.Count','SYN.Flag.Count', 'RST.Flag.Count', 'PSH.Flag.Count', 'ACK.Flag.Count', 
     'URG.Flag.Count', 'ECE.Flag.Count', 'Down.Up.Ratio','Average.Packet.Size', 'Avg.Fwd.Segment.Size', 'Avg.Bwd.Segment.Size', 'Fwd.Header.Length.1', 
     'Subflow.Fwd.Packets', 'Subflow.Fwd.Bytes','Subflow.Bwd.Packets', 'Subflow.Bwd.Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 
     'min_seg_size_forward','Active.Mean', 'Active.Max', 'Active.Min', 'Idle.Mean', 'Idle.Max','Idle.Min']
'''
'''
x = ['ProtocolName_factor', 'Flow.ID_factor', 'SourceIP_Octet1_factor','SourceIP_Octet2_factor','SourceIP_Octet3_factor','SourceIP_Octet4_factor', 'Source.Port', 'DestinationIP_Octet1_factor','DestinationIP_Octet2_factor','DestinationIP_Octet3_factor','DestinationIP_Octet4_factor','Destination.Port', 'Timestamp_factor', 'Flow.Duration_factor','Total.Fwd.Packets', 'Total.Backward.Packets', 'Total.Length.of.Fwd.Packets', 'Total.Length.of.Bwd.Packets', 'Fwd.Packet.Length.Max', 'Fwd.Packet.Length.Min', 'Fwd.Packet.Length.Mean', 'Fwd.Packet.Length.Std', 'Bwd.Packet.Length.Max', 'Bwd.Packet.Length.Min', 'Bwd.Packet.Length.Mean','Bwd.Packet.Length.Std', 'Flow.Bytes.s', 'Flow.Packets.s',
'Flow.IAT.Mean', 'Flow.IAT.Std', 'Flow.IAT.Max', 'Flow.IAT.Min', 'Fwd.IAT.Total', 'Fwd.IAT.Mean', 'Fwd.IAT.Std', 'Fwd.IAT.Max', 'Fwd.IAT.Min', 'Bwd.IAT.Total', 'Bwd.IAT.Mean', 'Bwd.IAT.Std',  'Bwd.IAT.Max', 'Bwd.IAT.Min', 'Fwd.PSH.Flags', 'Fwd.Header.Length', 'Bwd.Header.Length', 'Fwd.Packets.s', 'Bwd.Packets.s','Min.Packet.Length', 'Max.Packet.Length', 'Packet.Length.Mean', 'Packet.Length.Std', 'Packet.Length.Variance', 'FIN.Flag.Count','SYN.Flag.Count', 'RST.Flag.Count', 'PSH.Flag.Count', 'ACK.Flag.Count', 'URG.Flag.Count', 'ECE.Flag.Count', 'Down.Up.Ratio','Average.Packet.Size', 'Avg.Fwd.Segment.Size', 'Avg.Bwd.Segment.Size', 'Fwd.Header.Length.1', 'Subflow.Fwd.Packets', 'Subflow.Fwd.Bytes','Subflow.Bwd.Packets', 'Subflow.Bwd.Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward','Active.Mean', 'Active.Max', 'Active.Min', 'Idle.Mean', 'Idle.Max',
'Idle.Min', 'L7Protocol']

x = ['L7Protocol', 'Flow.ID_factor', 'SourceIP_Octet1_factor','SourceIP_Octet2_factor','SourceIP_Octet3_factor','SourceIP_Octet4_factor', 'Source.Port', 'DestinationIP_Octet1_factor','DestinationIP_Octet2_factor','DestinationIP_Octet3_factor','DestinationIP_Octet4_factor','Destination.Port']
'''
x=['class','min_fpktl', 'mean_fpktl', 'max_fpktl', 'std_fpktl', 'min_bpktl',
       'mean_bpktl', 'max_bpktl', 'std_bpktl', 'min_fiat', 'mean_fiat',
       'max_fiat', 'std_fiat', 'min_biat', 'mean_biat', 'max_biat',
       'std_biat', 'duration', 'proto', 'total_fpackets', 'total_fvolume',
       'total_bpackets', 'total_bvolume']

X=data.reindex(columns=x[1:])
X =  X[[col for col in X if X[col].nunique() > 1]]
#print(X.head(5))

# Preprocess the data to normalize the dataset
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result
X = normalize(X)
#print(X.head(5))
#print(data.head())

#Y = data['class']
Y = pd.DataFrame(columns=['class_factor'])
#print(Y)
Y['class_factor']=data['class']
Y['class_factor']=Y['class_factor'].factorize()[0]
print(Y['class_factor'])
print(Y.columns.values)
#Check correlation between the target column and other columns
z = X
z_colnames = z.columns.values.tolist()
z_colnames.append('class_factor')
z = z.reindex(columns=z_colnames)
#print(z)
z['class_factor'] = Y
#print(z)
corr_matrix = abs(z.corr(method='pearson'))
#print(corr_matrix)
y=corr_matrix['class_factor'].sort_values(ascending=False)
print('Correlation between column X and column \'class_factor\'\n',y)
print(X.head())

0        0
1        1
2        2
3        3
4        4
5        5
6        6
7        7
8        8
9        9
10      10
11       6
12       6
13       6
14       6
15       6
16       6
17       6
18       6
19       6
20       6
21       6
22       6
23       6
24       6
25       6
26       6
27       6
28       6
29       6
        ..
9970     4
9971     4
9972     4
9973     4
9974     4
9975     4
9976     4
9977     4
9978     4
9979     4
9980     4
9981     4
9982     4
9983     4
9984     4
9985     4
9986     4
9987     4
9988     4
9989     4
9990     4
9991     4
9992     4
9993     4
9994     4
9995     4
9996     4
9997     4
9998     4
9999     4
Name: class_factor, Length: 10000, dtype: int64
['class_factor']
Correlation between column X and column 'class'
 class_factor      1.000000
max_fpktl         0.953413
std_fpktl         0.952275
mean_fpktl        0.952125
proto             0.886358
max_biat          0.845367
max_fiat          0.803077
max_bpktl         0.772896

In [ ]:
# May drop the columns with low correlation
'''
Column_drop = ['Average.Packet.Size','Flow.IAT.Mean','SYN.Flag.Count','Fwd.PSH.Flags','Idle.Max','Idle.Mean','Idle.Min','Flow.ID_factor','Bwd.IAT.Total',
               'Fwd.Packet.Length.Min','Min.Packet.Length','Fwd.Packet.Length.Max','Fwd.Packet.Length.Std','Avg.Fwd.Segment.Size','Fwd.Packet.Length.Mean',
               'Flow.IAT.Min','Fwd.IAT.Min','Subflow.Fwd.Bytes','Total.Length.of.Fwd.Packets','Bwd.IAT.Max','Active.Mean','Active.Min','Bwd.IAT.Min',
               'Fwd.Header.Length.1','Fwd.Header.Length','Active.Max','RST.Flag.Count','ECE.Flag.Count','FIN.Flag.Count',
               'Bwd.IAT.Mean','Subflow.Fwd.Packets','Total.Fwd.Packets','Bwd.Packet.Length.Min','Bwd.IAT.Std','Bwd.Packets.s',
               'Timestamp_factor','Init_Win_bytes_backward','act_data_pkt_fwd']
'''
#Column_drop=[col in Y if Y[col][1]<0.3]
#print(Column_drop)
#Column_drop=['DestinationIP_2_factor','Destination.Port','ACK.Flag.Count','DestinationIP_1_factor','PSH.Flag.Count']
#Column_drop.extend(['URG.Flag.Count','SourceIP_3_factor','Flow.Duration_factor','SourceIP_27_factor','DestinationIP_3_factor','SourceIP_28_factor','Down.Up.Ratio','SourceIP_31_factor','SourceIP_30_factor','SourceIP_29_factor','SourceIP_26_factor','SourceIP_4_factor'])
#Column_drop.extend(['SourceIP_25_factor','SourceIP_2_factor','Bwd.Packet.Length.Std','SourceIP_20_factor','SourceIP_21_factor','SourceIP_19_factor','SourceIP_22_factor','Bwd.Packet.Length.Max','SourceIP_5_factor','SourceIP_23_factor','SourceIP_24_factor','Init_Win_bytes_forward','Fwd.Packets.s','Flow.Packets.s','Avg.Bwd.Segment.Size','Bwd.Packet.Length.Mean'])
#Column_drop.extend(['SourceIP_18_factor','Packet.Length.Std','Max.Packet.Length','SourceIP_6_factor','SourceIP_17_factor','DestinationIP_31_factor','DestinationIP_29_factor','DestinationIP_30_factor','SourceIP_7_factor','Fwd.IAT.Total','DestinationIP_28_factor','Fwd.IAT.Max','DestinationIP_27_factor'])
#print(y[y[:]>0.5])




In [ ]:
#X=X.drop(columns=Column_drop)
X=X[Column_drop]
# Split the datase into training and testing dataset
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X,Y, data.index, test_size=0.2, random_state=0)
print('Data size of training set:',len(X_train))
print('Data size of testing set:',len(X_test))
print(X_train.head())

Data size of training set: 8000
Data size of testing set: 2000
      DestinationIP_2_factor  Destination.Port  ACK.Flag.Count  \
7389                1.000000          0.000000             0.0   
9275                0.666667          0.893047             1.0   
2995                0.000000          0.046617             0.0   
5316                0.000000          0.046617             1.0   
356                 0.666667          0.783984             1.0   

      DestinationIP_1_factor  PSH.Flag.Count  URG.Flag.Count  \
7389                     0.0             1.0             0.0   
9275                     1.0             0.0             1.0   
2995                     0.0             1.0             0.0   
5316                     0.0             0.0             0.0   
356                      1.0             0.0             0.0   

      SourceIP_3_factor  Flow.Duration_factor  SourceIP_27_factor  \
7389           0.166667              0.741650            0.077465   
9275           0.

In [ ]:
from sklearn.svm import LinearSVC
#Linear SVC
lsvc = LinearSVC()
y_pred_LSVC = lsvc.fit(X_train, y_train).predict(X_test)
print("Accuracy of SVC: ", metrics.accuracy_score(y_test,y_pred_LSVC)*100)

#KNN
knn = KNeighborsClassifier()
y_pred_knn = knn.fit(X_train, y_train).predict(X_test)
print("Accuracy of KNN:",metrics.accuracy_score(y_test, y_pred_knn)*100)


#DTC
from sklearn.tree import DecisionTreeClassifier
clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 0)
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 0)
y_pred_DTC_gini = clf_gini.fit(X_train, y_train).predict(X_test)
y_pred_DTC_entropy = clf_entropy.fit(X_train, y_train).predict(X_test)
print("Accuracy of DTC (gini): ", metrics.accuracy_score(y_test,y_pred_DTC_gini)*100)
print("Accuracy of DTC (entropy):", metrics.accuracy_score(y_test,y_pred_DTC_entropy)*100)

#RFC
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=2000, random_state=0)
y_pred_rfc = rfc.fit(X_train, y_train).predict(X_test)
print("Accuracy of Random Forest Classifier:",metrics.accuracy_score(y_test, y_pred_rfc)*100)

from sklearn import metrics

# Import Support Vector Classifier
from sklearn.svm import SVC
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
svc=SVC(probability=True, kernel='linear')

#NaiveBayes

#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#from sklearn.feature_selection import RFE

from sklearn.naive_bayes import MultinomialNB
mnb_model=MultinomialNB()
#rfe=RFE(mnb_model,10)
trained_model=mnb_model.fit(X_train,y_train)
predicted_model=trained_model.predict(X_test)
print("Accuracy of Multinomial Naive Bayes:",metrics.accuracy_score(y_test, predicted_model)*100)

from sklearn.naive_bayes import BernoulliNB
bnb_model=BernoulliNB()
trained_modelb=bnb_model.fit(X_train,y_train)
predicted_modelb=trained_modelb.predict(X_test)
print("Accuracy of Bernoulli Naive Bayes:",metrics.accuracy_score(y_test, predicted_modelb)*100)

from sklearn.naive_bayes import GaussianNB
gnb_model=GaussianNB()

trained_modelg=gnb_model.fit(X_train,y_train)
predicted_modelg=trained_modelg.predict(X_test)
print("Accuracy of Gaussian Naive Bayes:",metrics.accuracy_score(y_test, predicted_modelg)*100)

from sklearn.ensemble import AdaBoostClassifier
boost = AdaBoostClassifier(n_estimators=200, base_estimator=svc, learning_rate=1)
#Train Adaboost Classifer
model_ada = boost.fit(X_train, y_train)
#Predict the response for test dataset
y_pred = model_ada.predict(X_test)
print("Accuracy of AdaBoost Classifer:",metrics.accuracy_score(y_test, y_pred)*100)

from xgboost import XGBClassifier
xgmodel = XGBClassifier()
xgmodel.fit(X_train, y_train)
# make predictions for test data
y_pred = xgmodel.predict(X_test)
print("Accuracy of XGBoost Classifer:",metrics.accuracy_score(y_test, y_pred)*100)


Accuracy of SVC:  65.60000000000001
Accuracy of KNN: 77.60000000000001
Accuracy of DTC (gini):  82.19999999999999
Accuracy of DTC (entropy): 82.6
Accuracy of Random Forest Classifier: 88.55
Accuracy of Multinomial Naive Bayes: 54.35
Accuracy of Bernoulli Naive Bayes: 50.949999999999996
Accuracy of Gaussian Naive Bayes: 45.0


In [ ]:
# DTC gini decision tree image:
plt.rcParams['figure.figsize'] = [20,20]
from sklearn.tree import export_graphviz
from sklearn import tree
from IPython.display import SVG
from graphviz import Source
from IPython.display import display
graph = Source(tree.export_graphviz(clf_gini.fit(X_train, y_train), out_file=None, filled = True))
display(SVG(graph.pipe(format='svg')))

In [ ]:
# Saving the model

import joblib
model = rfc.fit(X_train, y_train)
filename = 'finalized_model.sav'
joblib.dump(model, filename)